In [12]:
import os
import pandas as pd
import numpy as np

topics_init = [x for x in os.listdir("./bbc-fulltext/bbc") if x != "README.TXT"]

In [24]:
article_list = {"heading":[], "topic":[]}

for x in topics_init:
    for article_title in os.listdir(os.path.join("./bbc-fulltext/bbc", x)):
        with open(os.path.join("./bbc-fulltext/bbc", x, article_title), "r") as f:
            heading = f.readline()
            article_list["heading"].append(heading.split("\n")[0])
            article_list["topic"].append(x)
article_df = pd.DataFrame(article_list)
article_df["heading"].head()

0    Ad sales boost Time Warner profit
1     Dollar gains on Greenspan speech
2    Yukos unit buyer faces loan claim
3    High fuel prices hit BA's profits
4    Pernod takeover talk lifts Domecq
Name: heading, dtype: object

In [31]:
from keras.src.utils import pad_sequences
from keras.src.preprocessing.text import Tokenizer

headings_arr = article_df["heading"].values
heading_tokenizer = Tokenizer(oov_token="<OOV>")
heading_tokenizer.fit_on_texts(headings_arr)

sequenced_headings = heading_tokenizer.texts_to_sequences(headings_arr)
padded_heading = pad_sequences(sequenced_headings, padding="post", truncating="post", maxlen = 10)

Ad sales boost Time Warner profit


In [33]:
from sklearn.preprocessing import LabelEncoder

topic_encoder = LabelEncoder()
ys = topic_encoder.fit_transform(article_df["topic"].values)
ys[:5]

array([0, 0, 0, 0, 0])

In [34]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(padded_heading, ys, train_size=0.8, random_state=4)

In [55]:
from keras.src.layers import Embedding, Bidirectional, Dropout, LSTM, Conv1D, GlobalAveragePooling1D, Dense
from keras import Sequential

model = Sequential([
    Embedding(len(heading_tokenizer.word_index)+1, 32, input_length = 10),
    Dropout(0.2),
    Conv1D(64, 4, activation="relu"),
    # Bidirectional(LSTM(32)),
    GlobalAveragePooling1D(),
    Dense(32, activation="relu"),
    Dense(5, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [56]:
model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
56/56 [==============================] - 2s 8ms/step - loss: 1.6010 - accuracy: 0.2449 - val_loss: 1.5863 - val_accuracy: 0.2360
Epoch 2/20
56/56 [==============================] - 0s 6ms/step - loss: 1.5024 - accuracy: 0.3472 - val_loss: 1.4095 - val_accuracy: 0.4045
Epoch 3/20
56/56 [==============================] - 0s 6ms/step - loss: 1.0754 - accuracy: 0.5556 - val_loss: 1.1409 - val_accuracy: 0.5079
Epoch 4/20
56/56 [==============================] - 0s 6ms/step - loss: 0.6326 - accuracy: 0.8157 - val_loss: 0.9455 - val_accuracy: 0.6764
Epoch 5/20
56/56 [==============================] - 0s 6ms/step - loss: 0.2935 - accuracy: 0.9472 - val_loss: 0.7997 - val_accuracy: 0.7236
Epoch 6/20
56/56 [==============================] - 0s 6ms/step - loss: 0.1165 - accuracy: 0.9837 - val_loss: 0.7660 - val_accuracy: 0.7438
Epoch 7/20
56/56 [==============================] - 0s 6ms/step - loss: 0.0489 - accuracy: 0.9972 - val_loss: 0.7588 - val_accuracy: 0.7506
Epoch 8/20
56/56 [==

In [77]:
padded_input = pad_sequences(heading_tokenizer.texts_to_sequences([""]), maxlen=10, padding="post", truncating="post")
prediction = model.predict(padded_input)
print(topic_encoder.inverse_transform([np.argmax(prediction)]))

1/1 [==============================] - 0s 26ms/step
['politics']
